In [1]:
# !wget -O face_landmarker_v2_with_blendshapes.task -q https://storage.googleapis.com/mediapipe-models/face_landmarker/face_landmarker/float16/1/face_landmarker.task

In [2]:
import mediapipe as mp # type: ignore
from mediapipe import solutions # type: ignore
from mediapipe.framework.formats import landmark_pb2 # type: ignore
import numpy as np # type: ignore
import matplotlib.pyplot as plt # type: ignore
import time # type: ignore

def draw_landmarks_on_image(rgb_image, detection_result):
  face_landmarks_list = detection_result.face_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected faces to visualize
  for idx in range(len(face_landmarks_list)):
    face_landmarks = face_landmarks_list[idx]

    # Draw the face landmarks
    face_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    face_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in face_landmarks
    ])

    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_TESSELATION,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_tesselation_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_CONTOURS,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_contours_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_IRISES,
          landmark_drawing_spec=None,
          connection_drawing_spec=mp.solutions.drawing_styles
          .get_default_face_mesh_iris_connections_style())

  return annotated_image

def plot_face_blendshapes_bar_graph(face_blendshapes):
  # Extract the face blendshapes category names and scores.
  face_blendshapes_names = [face_blendshapes_category.category_name for face_blendshapes_category in face_blendshapes]
  face_blendshapes_scores = [face_blendshapes_category.score for face_blendshapes_category in face_blendshapes]
  # The blendshapes are ordered in decreasing score value.
  face_blendshapes_ranks = range(len(face_blendshapes_names))

  _, ax = plt.subplots(figsize=(12, 12))
  bar = ax.barh(face_blendshapes_ranks, face_blendshapes_scores, label=[str(x) for x in face_blendshapes_ranks])
  ax.set_yticks(face_blendshapes_ranks, face_blendshapes_names)
  ax.invert_yaxis()

  # Label each bar with values
  for score, patch in zip(face_blendshapes_scores, bar.patches):
    plt.text(patch.get_x() + patch.get_width(), patch.get_y(), f"{score:.4f}", va="top")

  ax.set_xlabel('Score')
  ax.set_title("Face Blendshapes")
  plt.tight_layout()
  plt.show()

In [3]:
last_is_blink = False
last_doze_time = 0
last_wake_time = 0
doze_counter = 0
doze_timer = 0
doze_start = 0
is_doze = False
def print_eyes_status(face_blendshapes):
  global last_is_blink
  global last_doze_time
  global last_wake_time
  global doze_counter
  global doze_timer
  global doze_start
  global is_doze
  face_blendshapes_scores = np.array([face_blendshapes_category.score for face_blendshapes_category in face_blendshapes])
  upward_score = face_blendshapes_scores[17] + face_blendshapes_scores[18]
  downward_score = face_blendshapes_scores[11] + face_blendshapes_scores[12]
  leftward_score = face_blendshapes_scores[14] + face_blendshapes_scores[15]
  rightward_score = face_blendshapes_scores[13] + face_blendshapes_scores[16]
  dir_score = np.array([["upward",upward_score], ["downward",downward_score], ["leftward",leftward_score], ["rightward",rightward_score]])
  dir = dir_score[np.argmax(dir_score[:,1])][0]
  #print(dir)
  is_blink = face_blendshapes_scores[9]>0.6 and face_blendshapes_scores[10]>0.6
  if is_blink:
    if not last_is_blink:
      last_doze_time = time.time()
    if last_doze_time != 0 and time.time() > last_doze_time:
      doze_counter += 1
      time.sleep(0.01)
    if doze_counter > 70:
      if not is_doze:
        doze_start = time.time()
      is_doze = True
      last_wake_time = 0

  else:
    if last_is_blink:
      last_wake_time = time.time()
    if last_wake_time != 0 and time.time() > last_wake_time:
      doze_counter -= 1
      time.sleep(0.01)
    if doze_counter < 20:
      doze_counter = 0
      is_doze = False
  doze_timer = time.time() - doze_start    
  last_is_blink = is_blink
  #print(is_blink)
  if is_doze:
    print(f"doze counter {doze_counter}",f"dozing for {doze_timer:.2f}s")
  else:
    print(f"{'blinking' if is_blink else f'looking {dir}'}")

In [4]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

def print_hands_status(hands_results):
    left_hand_x = np.array([])
    left_hand_y = np.array([])
    right_hand_x = np.array([])
    right_hand_y = np.array([])
    mp_hands = mp.solutions.hands
    if len(hands_results.multi_handedness) == 1:
      if (hands_results.multi_handedness[0].classification[0].index == 0 and hands_results.multi_handedness[0].classification[0].label == "Left"):
        for handLandmarks in hands_results.multi_hand_landmarks:
          for point in mp_hands.HandLandmark:
            left_hand_x = np.append(left_hand_x, handLandmarks.landmark[point].x)
            left_hand_y = np.append(left_hand_y, handLandmarks.landmark[point].y)
        left_hand_mean_x = np.mean(left_hand_x)
        left_hand_mean_y = np.mean(left_hand_y)
        # print("left hand:", left_hand_mean_x, left_hand_mean_y)
        print('\033[2;31;42m [WARNING] \033[0;0m',"right hand is not detected")
        if (not 0.25<left_hand_mean_x<0.75 or not left_hand_mean_y>0.6):
          print('\033[2;31;43m [WARNING] \033[0;0m',"left hand is out of the zone")
        
      elif (hands_results.multi_handedness[0].classification[0].index == 1 and hands_results.multi_handedness[0].classification[0].label == "Right"):
        for handLandmarks in hands_results.multi_hand_landmarks:
          for point in mp_hands.HandLandmark:
            right_hand_x = np.append(right_hand_x, handLandmarks.landmark[point].x)
            right_hand_y = np.append(right_hand_y, handLandmarks.landmark[point].y)
        right_hand_mean_x = np.mean(right_hand_x)
        right_hand_mean_y = np.mean(right_hand_y)
        # print("right hand:", right_hand_mean_x, right_hand_mean_y)
        print('\033[2;31;42m [WARNING] \033[0;0m',"left hand is not detected")
        if (not 0.25<right_hand_mean_x<0.75 or not right_hand_mean_y>0.6):
          print('\033[2;31;43m [WARNING] \033[0;0m',"right hand is out of the zone")

    elif len(hands_results.multi_handedness) == 2:
      if (hands_results.multi_handedness[0].classification[0].index == 0 and hands_results.multi_handedness[0].classification[0].label == "Left"):
        for point in mp_hands.HandLandmark:
          left_hand_x = np.append(left_hand_x, hands_results.multi_hand_landmarks[0].landmark[point].x)
          left_hand_y = np.append(left_hand_y, hands_results.multi_hand_landmarks[0].landmark[point].y)
          right_hand_x = np.append(right_hand_x, hands_results.multi_hand_landmarks[1].landmark[point].x)
          right_hand_y = np.append(right_hand_y, hands_results.multi_hand_landmarks[1].landmark[point].y)
      
      else:
        for point in mp_hands.HandLandmark:
          left_hand_x = np.append(left_hand_x, hands_results.multi_hand_landmarks[1].landmark[point].x)
          left_hand_y = np.append(left_hand_y, hands_results.multi_hand_landmarks[1].landmark[point].y)
          right_hand_x = np.append(right_hand_x, hands_results.multi_hand_landmarks[0].landmark[point].x)
          right_hand_y = np.append(right_hand_y, hands_results.multi_hand_landmarks[0].landmark[point].y)

      left_hand_mean_x = np.mean(left_hand_x)
      left_hand_mean_y = np.mean(left_hand_y)
      right_hand_mean_x = np.mean(right_hand_x)
      right_hand_mean_y = np.mean(right_hand_y)
      if ((not 0.25<left_hand_mean_x<0.75 or not left_hand_mean_y>0.6) and (not 0.25<right_hand_mean_x<0.75 or not right_hand_mean_y>0.6)):
        print('\033[2;31;43m [WARNING] \033[0;0m',"both hands are out of the zone"),print()
      elif (not 0.25<left_hand_mean_x<0.75 or not left_hand_mean_y>0.6):
        print('\033[2;31;43m [WARNING] \033[0;0m',"left hand is out of the zone")
      elif (not 0.25<right_hand_mean_x<0.75 or not right_hand_mean_y>0.6):
        print('\033[2;31;43m [WARNING] \033[0;0m',"right hand is out of the zone")
      else:
        print("both hands are in the zone")

    
  

In [5]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

base_options = python.BaseOptions(model_asset_path='face_landmarker_v2_with_blendshapes.task')
options = vision.FaceLandmarkerOptions(base_options=base_options,
                                       output_face_blendshapes=True,
                                       output_facial_transformation_matrixes=True,
                                       num_faces=1)
faces = vision.FaceLandmarker.create_from_options(options)

In [6]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

# For webcam input:
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

with mp_hands.Hands(
    model_complexity=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
  
  while cap.isOpened():
    if cv2.waitKey(5) & 0xFF == ord('q'):  # press q to quit
      print("Exiting...")          
      cap.release()
      cv2.destroyAllWindows()
      break
    success, frame = cap.read()
    if not success:
      print("ignoring empty camera frame")
      # If loading a video, use 'break' instead of 'continue'.
      continue
    frame = cv2.flip(frame, 1)
    raw_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    cam_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)

    face_results = faces.detect(cam_image)

    hands_results = hands.process(raw_image)

    if len(face_results.face_blendshapes)==0 or len(face_results.face_landmarks)==0:
      print('\033[2;31;46m [WARNING] \033[0;0m',"No face detected")
      continue
    if hands_results == None or hands_results.multi_hand_landmarks == None:
      print('\033[2;31;46m [WARNING] \033[0;0m',"No hands detected")
      continue
    # draw face landmarks
    annotated_image = draw_landmarks_on_image(cam_image.numpy_view(), face_results)
    
    # draw hands landmarks
    if hands_results.multi_hand_landmarks:
      for hand_landmarks in hands_results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(
            annotated_image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())
        
    cv2.namedWindow('cam', cv2.WINDOW_AUTOSIZE)
    cv2.imshow("cam", annotated_image)

    # print the status of eyes and hands
    print_eyes_status(face_results.face_blendshapes[0])
    print_hands_status(hands_results)


c:\Users\student\.conda\envs\mediapipe\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


 [WARNING]  No hands detected
 [WARNING]  No hands detected
 [WARNING]  No hands detected
 [WARNING]  No hands detected
 [WARNING]  No hands detected
 [WARNING]  No hands detected
 [WARNING]  No hands detected
 [WARNING]  No hands detected
 [WARNING]  No hands detected
 [WARNING]  No hands detected
 [WARNING]  No hands detected
 [WARNING]  No hands detected
 [WARNING]  No hands detected
 [WARNING]  No hands detected
 [WARNING]  No hands detected
 [WARNING]  No hands detected
 [WARNING]  No hands detected
 [WARNING]  No hands detected
 [WARNING]  No hands detected
 [WARNING]  No hands detected
 [WARNING]  No hands detected
 [WARNING]  No hands detected
 [WARNING]  No hands detected
 [WARNING]  No hands detected
 [WARNING]  No hands detected
 [WARNING]  No hands detected
 [WARNING]  No hands detected
 [WARNING]  No hands detected
 [WARNING]  No hands detected
 [WARNING]  No hands detected
 [WARNING]  No hands detected
 [WARNING]  No hands detected
 [WARNING]  No hands detected
 [WARNING]